**Материалы**
У вас лекционный ноутбук в архиве он с картинками и презентация тоже есть в архиве

**Задание**
На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
4. сдлать выводы что получилось лучше 

In [19]:
#!pip install stop_words
#!pip install pymorphy2

In [20]:
import pandas as pd

df = pd.read_csv("отзывы за лето.csv", encoding='cp1251')

In [21]:
len(df)

20659

In [22]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [23]:
#!pip install tensorflow

In [24]:
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
#import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, Conv2D, GlobalMaxPool1D, concatenate, Flatten, add, MaxPool1D, RepeatVector
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Masking, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, TimeDistributed, AveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten, SimpleRNN, LSTM, GRU, Reshape
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy


from sklearn.model_selection import train_test_split
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [25]:
stopwordslist = stopwords.words("russian")
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

morpher = MorphAnalyzer()

def words_only(text):
    text=str(text)
    return text.lower()   

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [26]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kote\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kote\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
df['normalized_content'] = df['Content'].apply(normalize_text)
df.head()

,Rating,Content,Date,normalized_content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать 1 работа антивирус далёкий нику...
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро


In [28]:
max_words = 20000
max_len = 150
num_classes = 5

epochs = 20
batch_size = 512
print_batch_n = 100

In [29]:
train_corpus = " ".join(df['normalized_content'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [30]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
voc = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [31]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

data_train = np.asarray(
    [text_to_sequence(text, max_len) for text in df['normalized_content']], 
    dtype=np.int32)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(data_train, df.Rating, test_size=0.3, random_state=1)

le = LabelEncoder()

y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)

In [33]:
#%load_ext tensorboard
#%tensorboard --logdir logs

In [34]:
model_name = 'RNN'
model = Sequential()
model.add(
    Embedding(input_dim=max_words, #это размер словаря в текстовых данных
              input_length=max_len, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=128, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(SimpleRNN(32, recurrent_dropout=0.2, return_sequences=True)) #возвращать всю последовательность выходных данных для каждого элемента (по одному вектору на каждый шаг)
model.add(SimpleRNN(32, recurrent_dropout=0.2))#отбрасывание линейных значений повторов 0.2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam', loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться 


tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name, 
    write_graph=False, update_freq=100, profile_batch=0)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping]) 

Epoch 1/10
26/26 [==============================] - 14s 418ms/step - loss: 1.2871 - accuracy: 0.5230 - val_loss: 0.9385 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 11s 406ms/step - loss: 0.9673 - accuracy: 0.7017 - val_loss: 0.8032 - val_accuracy: 0.7270
Epoch 3/10
26/26 [==============================] - 10s 401ms/step - loss: 0.8475 - accuracy: 0.7192 - val_loss: 0.7653 - val_accuracy: 0.7408
Epoch 4/10
26/26 [==============================] - 11s 406ms/step - loss: 0.7724 - accuracy: 0.7370 - val_loss: 0.7150 - val_accuracy: 0.7491
Epoch 5/10
26/26 [==============================] - 11s 407ms/step - loss: 0.7150 - accuracy: 0.7552 - val_loss: 0.6787 - val_accuracy: 0.7595
Epoch 6/10
26/26 [==============================] - 10s 401ms/step - loss: 0.6735 - accuracy: 0.7706 - val_loss: 0.6655 - val_accuracy: 0.7678
Epoch 7/10
26/26 [==============================] - 10s 404ms/step - loss: 0.6410 - accuracy: 0.7775 - val_loss: 0.6656 - val_accuracy: 0.7664

In [35]:
score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)

metrics_df = pd.DataFrame(columns=['model', 'Test score', 'Test accuracy'])

metrics_df = metrics_df.append({
    'model': 'RNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)


print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 95ms/step - loss: 0.6829 - accuracy: 0.7585


Test score: 0.6828565001487732
Test accuracy: 0.7584704756736755


C:\Users\kote\AppData\Local\Temp\ipykernel_8812\2619252210.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


In [36]:
model_name = 'CNN_RNN'
model = Sequential()

model.add(
    Embedding(input_dim=max_words, #это размер словаря в текстовых данных
              input_length=max_len, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=128, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(Conv1D(128, 3, activation='relu', padding="same"))
model.add(SimpleRNN(64, recurrent_dropout=0.2)) #отбрасывание линейных значений повторов 0.2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam', loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться 

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name, 
    write_graph=False, update_freq=100, profile_batch=0)


In [37]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
26/26 [==============================] - 17s 574ms/step - loss: 1.2410 - accuracy: 0.5394 - val_loss: 0.8795 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 15s 571ms/step - loss: 0.8529 - accuracy: 0.7195 - val_loss: 0.7500 - val_accuracy: 0.7464
Epoch 3/10
26/26 [==============================] - 15s 562ms/step - loss: 0.7396 - accuracy: 0.7504 - val_loss: 0.6962 - val_accuracy: 0.7671
Epoch 4/10
26/26 [==============================] - 15s 574ms/step - loss: 0.6652 - accuracy: 0.7771 - val_loss: 0.6915 - val_accuracy: 0.7678
Epoch 5/10
26/26 [==============================] - 14s 548ms/step - loss: 0.6105 - accuracy: 0.7906 - val_loss: 0.7331 - val_accuracy: 0.7609


In [38]:
score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)

metrics_df = metrics_df.append({
    'model': 'CNN_RNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 146ms/step - loss: 0.7382 - accuracy: 0.7596


Test score: 0.7382273077964783
Test accuracy: 0.7595998644828796


C:\Users\kote\AppData\Local\Temp\ipykernel_8812\1873839954.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


In [40]:
model_name = 'RNN_CNN'
model = Sequential()

model.add(
    Embedding(input_dim=max_words, #это размер словаря в текстовых данных
              input_length=max_len, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=64, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(SimpleRNN(64, recurrent_dropout=0.2, return_sequences=True)) #отбрасывание линейных значений повторов 0.2
# model.add(RepeatVector(32))
model.add(Conv1D(128, 3, activation='relu', padding="same"))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam', loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться 

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name, 
    write_graph=False, update_freq=100, profile_batch=0)


In [41]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
26/26 [==============================] - 15s 500ms/step - loss: 1.3018 - accuracy: 0.6544 - val_loss: 1.1817 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 13s 498ms/step - loss: 1.0230 - accuracy: 0.7037 - val_loss: 0.9151 - val_accuracy: 0.7091
Epoch 3/10
26/26 [==============================] - 13s 489ms/step - loss: 0.8681 - accuracy: 0.7122 - val_loss: 0.8063 - val_accuracy: 0.7097
Epoch 4/10
26/26 [==============================] - 13s 484ms/step - loss: 0.7831 - accuracy: 0.7252 - val_loss: 0.7838 - val_accuracy: 0.7208
Epoch 5/10
26/26 [==============================] - 13s 485ms/step - loss: 0.7236 - accuracy: 0.7335 - val_loss: 0.6935 - val_accuracy: 0.7588
Epoch 6/10
26/26 [==============================] - 13s 486ms/step - loss: 0.6872 - accuracy: 0.7741 - val_loss: 0.7035 - val_accuracy: 0.7692


In [42]:
score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
metrics_df = metrics_df.append({
    'model': 'RNN_CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 127ms/step - loss: 0.7163 - accuracy: 0.7536


Test score: 0.716271162033081
Test accuracy: 0.7536302208900452


C:\Users\kote\AppData\Local\Temp\ipykernel_8812\799318069.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


In [43]:
model_name = 'LSTM'
model = Sequential()

model.add(
    Embedding(input_dim=max_words, #это размер словаря в текстовых данных
              input_length=max_len, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=30, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(LSTM(64, recurrent_dropout=0.2)) #отбрасывание линейных значений повторов 0.2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam', loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться 

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name, 
    write_graph=False, update_freq=100, profile_batch=0)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
26/26 [==============================] - 18s 579ms/step - loss: 1.3577 - accuracy: 0.6561 - val_loss: 0.9786 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 16s 637ms/step - loss: 0.9866 - accuracy: 0.7044 - val_loss: 0.8698 - val_accuracy: 0.7091
Epoch 3/10
26/26 [==============================] - 19s 717ms/step - loss: 0.8337 - accuracy: 0.7170 - val_loss: 0.7411 - val_accuracy: 0.7436
Epoch 4/10
26/26 [==============================] - 21s 794ms/step - loss: 0.7285 - accuracy: 0.7569 - val_loss: 0.6862 - val_accuracy: 0.7581
Epoch 5/10
26/26 [==============================] - 21s 810ms/step - loss: 0.6608 - accuracy: 0.7711 - val_loss: 0.6747 - val_accuracy: 0.7685
Epoch 6/10
26/26 [==============================] - 22s 829ms/step - loss: 0.6269 - accuracy: 0.7819 - val_loss: 0.6510 - val_accuracy: 0.7740
Epoch 7/10
26/26 [==============================] - 22s 853ms/step - loss: 0.5943 - accuracy: 0.7902 - val_loss: 0.6468 - val_accuracy: 0.7795

In [44]:
score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)

metrics_df = metrics_df.append({
    'model': 'LSTM',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 3s 192ms/step - loss: 0.6703 - accuracy: 0.7620


Test score: 0.670281171798706
Test accuracy: 0.7620199918746948


C:\Users\kote\AppData\Local\Temp\ipykernel_8812\3884853795.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


In [45]:
metrics_df.sort_values(by='Test score', ascending=False)

,model,Test score,Test accuracy
1,CNN_RNN,0.738227,0.75960
2,RNN_CNN,0.716271,0.75363
0,RNN,0.682857,0.75847
3,LSTM,0.670281,0.76202
